# Introduction to TensorFlow

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.layers as tfl
from pprint import pprint

In [9]:
print(tf.__version__)

2.13.0


## Constants

In [10]:
scalar = tf.constant(1, dtype=tf.int8)
vector = tf.constant([1, 2, 3], dtype=tf.float32)
matrix = tf.constant(np.array([[1, 2], [3, 4]]), dtype=tf.float32)

pprint([scalar, vector, matrix])

[<tf.Tensor: shape=(), dtype=int8, numpy=1>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>]


## Variables
Tensors are immutable, the values in tensors can't be updated in-place. Variables are just like Tensors in terms of as inputs to operations, but with the in-place value update methods.

In [12]:
x = tf.random.uniform((3, 2))
v = tf.Variable(x)
pprint(v)

<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[0.48382032, 0.7403233 ],
       [0.8386644 , 0.7816727 ],
       [0.95171857, 0.6495646 ]], dtype=float32)>


In [13]:
pprint(tf.square(v))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.2340821, 0.5480786],
       [0.703358 , 0.6110122],
       [0.9057682, 0.4219342]], dtype=float32)>


In [14]:
# Variables can be updated with .assign, .assign_add, .assign_sub
v.assign(tf.square(v))
pprint(v)

<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[0.2340821, 0.5480786],
       [0.703358 , 0.6110122],
       [0.9057682, 0.4219342]], dtype=float32)>


## Loading Data

In [18]:
housing_url = 'https://www.kaggle.com/datasets/camnugent/california-housing-prices'
path = tf.keras.utils.get_file(housing_url.split("/")[-1], housing_url)

   8192/Unknown - 0s 0us/step

In [19]:
def map_line(x):
    return tf.strings.to_number(tf.strings.split(x))

In [20]:
housing = (tf.data.TextLineDataset(path) .map(map_line))

In [24]:
print(housing)

<_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.float32, name=None)>


In [25]:
mnist = tfds.load('mnist', split=None)
mnist_train = mnist['train']
mnist_test = mnist['test']

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /Users/dmmil/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [31]:
x_vals = np.array([1., 3., 5., 7., 9.])
x_data = tf.Variable(x_vals, dtype=tf.float32)
m_const = tf.constant(3.)
operation = tf.multiply(x_data, m_const)
for result in operation:
    print(result)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(15.0, shape=(), dtype=float32)
tf.Tensor(21.0, shape=(), dtype=float32)
tf.Tensor(27.0, shape=(), dtype=float32)


In [2]:
# The adapt() method
data = np.array(
    [
        [0.1, 0.2, 0.3],
        [0.8, 0.9, 1.0],
        [1.5, 1.6, 1.7],
    ]
)
layer = tfl.Normalization()
layer.adapt(data)
normalized_data = layer(data)

print("Features mean: %.2f" % (normalized_data.numpy().mean()))
print("Features std: %.2f" % (normalized_data.numpy().std()))


Features mean: -0.00
Features std: 1.00


In [7]:
print(normalized_data.numpy())

[[-1.2247449e+00 -1.2247449e+00 -1.2247448e+00]
 [ 0.0000000e+00 -1.0428640e-07  0.0000000e+00]
 [ 1.2247449e+00  1.2247448e+00  1.2247449e+00]]


In [9]:
data = [
    "ξεῖν᾽, ἦ τοι μὲν ὄνειροι ἀμήχανοι ἀκριτόμυθοι",
    "γίγνοντ᾽, οὐδέ τι πάντα τελείεται ἀνθρώποισι.",
    "δοιαὶ γάρ τε πύλαι ἀμενηνῶν εἰσὶν ὀνείρων:",
    "αἱ μὲν γὰρ κεράεσσι τετεύχαται, αἱ δ᾽ ἐλέφαντι:",
    "τῶν οἳ μέν κ᾽ ἔλθωσι διὰ πριστοῦ ἐλέφαντος,",
    "οἵ ῥ᾽ ἐλεφαίρονται, ἔπε᾽ ἀκράαντα φέροντες:",
    "οἱ δὲ διὰ ξεστῶν κεράων ἔλθωσι θύραζε,",
    "οἵ ῥ᾽ ἔτυμα κραίνουσι, βροτῶν ὅτε κέν τις ἴδηται.",
]
layer = tfl.TextVectorization()
layer.adapt(data)
vectorized_text = layer(data)
print(vectorized_text.numpy())

[[37 12 25  5  9 20 21  0  0]
 [51 34 27 33 29 18  0  0  0]
 [49 52 30 31 19 46 10  0  0]
 [ 7  5 50 43 28  7 47 17  0]
 [24 35 39 40  3  6 32 16  0]
 [ 4  2 15 14 22 23  0  0  0]
 [36 48  6 38 42  3 45  0  0]
 [ 4  2 13 41 53  8 44 26 11]]


In [18]:
vocab = ["a", "b", "c", "d"]
data = tf.constant(
    [
        ["a", "c", "d"],
        ["d", "z", "b"]
    ]
)
layer = tfl.StringLookup(vocabulary=vocab)
vectorized_data = layer(data)
print(vectorized_data.numpy())

[[1 3 4]
 [4 0 2]]
